In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.edges.layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df  = pl.DataFrame(_lu_)
g      = rt.createNetworkXGraph(my_df, [('fm','to')])
if exists(layout_filename):
    my_pos = {}
    _rtg_ = rt.interactiveGraphLayout(my_df, ln_params={'relationships':[('fm','to')], 'pos':my_pos}, w=384, h=384)
    _rtg_.loadLayout(layout_filename)
    my_pos = _rtg_.pos
else:
    my_pos = nx.spring_layout(g)
    #pos = rt.hyperTreeLayout(g)
    #pos = nx.kamada_kawai_layout(g)
params = {'df':my_df, 'relationships':[('fm','to')], 'pos':my_pos} # , 'w':1280, 'h':1280}
# rt.chordDiagram(my_df, [('fm','to')], link_style='bundled', skeleton_algorithm='kmeans', link_opacity=0.05, w=512, h=512)

In [ ]:
def chordRoutedGraph(df, relationships, pos, k=6, iters=100, w=384, h=384, bounds_percent=0.2, neighborhood_opacity=0.2, draw_neighborhoods=True):
    # Create a link for transformation information
    _link_ = rt.link(df, relationships=relationships, pos=pos, w=w, h=h, bounds_percent=bounds_percent)
    _link_.renderSVG() # force the rendering and related transformations to occur

    # Flatten the nodes out -- assumes that all nodes (and only nodes) are in the pos dictionary
    _node_ls_  = list(pos.keys())
    _node_to_i_ = {}
    for i in range(len(_node_ls_)): _node_to_i_[_node_ls_[i]] = i
    _node_sxy_ = []
    for _node_ in _node_ls_: 
        sx, sy = _link_.xT(pos[_node_][0]), _link_.yT(pos[_node_][1])
        _node_sxy_.append((sx, sy))
    sx_min, sx_max, sy_min, sy_max = sx, sx, sy, sy
    for _node_ in _node_ls_:
        sx_min, sx_max = min(sx_min, sx), max(sx_max, sx)
        sy_min, sy_max = min(sy_min, sy), max(sy_max, sy)

    # Make random cluster centers
    cluster_centers = {}
    for i in range(k): 
        sx, sy = random.random() * (sx_max - sx_min) + sx_min, random.random() * (sy_max - sy_min) + sy_min
        cluster_centers[i] = (sx, sy)

    # Iterate K-Means
    for _iter_ in range(iters):
        # Assign nodes to their closest center
        center_assignments = {}
        for j in range(len(_node_ls_)):
            _node_ = _node_ls_[j]
            min_dist       = (_node_sxy_[j][0] - cluster_centers[0][0])**2 + (_node_sxy_[j][1] - cluster_centers[0][1])**2
            closest_center = 0
            for i in range(1, k):
                dist = (_node_sxy_[j][0] - cluster_centers[i][0])**2 + (_node_sxy_[j][1] - cluster_centers[i][1])**2
                if dist < min_dist:
                    min_dist       = dist
                    closest_center = i
            if closest_center not in center_assignments: center_assignments[closest_center] = []
            center_assignments[closest_center].append(_node_)
        # If there are any centers without nodes, assign a random node to them
        for i in range(k): 
            if i not in center_assignments: center_assignments[i] = [random.choice(_node_ls_)]
        # Update centers
        for i in range(k): 
            sx, sy = 0, 0
            for _node_ in center_assignments[i]: 
                sx, sy = sx + _node_sxy_[_node_to_i_[_node_]][0], sy + _node_sxy_[_node_to_i_[_node_]][1]
            cluster_centers[i] = (sx/len(center_assignments[i]), sy/len(center_assignments[i]))

    # Track nodes to their cluster centers
    node_to_cluster_center = {}
    for i in center_assignments:
        for _node_ in center_assignments[i]: 
            node_to_cluster_center[_node_] = i

    # Create the Voronoi diagram from the cluster centers
    voronoi_pts = []
    for i in cluster_centers: voronoi_pts.append(cluster_centers[i])
    nborhoods = rt.isedgarVoronoi(voronoi_pts, Box=[(0,_link_.h),(_link_.w,_link_.h),(_link_.w,0),(0,0)])

    # Create SVG
    distinguishable_colors = rt.co_mgr.brewerColors('qualitative', k)
    for i in range(k):
        rt.co_mgr.str_to_color_lu[i]  = distinguishable_colors[i]
        rt.co_mgr.str_to_color_lu[-i] = distinguishable_colors[i]
    svg = [f'<svg x="0" y="0" width="{w}" height="{h}">']

    if neighborhood_opacity != 1.0: svg.append(f'<g opacity="{neighborhood_opacity}">')
    voronoi_centers = {}
    for _poly_i_ in range(len(nborhoods)):
        _poly_ = nborhoods[_poly_i_]
        x_min, x_max, y_min, y_max = _poly_[0][0], _poly_[0][0], _poly_[0][1], _poly_[0][1]
        d      = f'M {_poly_[0][0]} {_poly_[0][1]}  '
        for i in range(1, len(_poly_)): 
            d += f'L {_poly_[i][0]} {_poly_[i][1]} '
            x_min, x_max = min(x_min, _poly_[i][0]), max(x_max, _poly_[i][0])
            y_min, y_max = min(y_min, _poly_[i][1]), max(y_max, _poly_[i][1])
        d += 'Z'
        voronoi_centers[_poly_i_] = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        if draw_neighborhoods: svg.append(f'<path d=\"{d}\" fill=\"{distinguishable_colors[_poly_i_]}\" />')
    if neighborhood_opacity != 1.0: svg.append('</g>')

    _radius_  = 64
    _circles_ = []
    for i in range(k): _circles_.append((voronoi_centers[i][0], voronoi_centers[i][1], _radius_))
    _circles_ = rt.crunchCircles(_circles_, min_d=40)


    neighborhood_to_diagram = {}
    _fm_, _to_ = relationships[0][0], relationships[0][1]
    for nborhood_i in center_assignments:
        in_nborhood = set(center_assignments[nborhood_i])
        _df_     = df.filter((pl.col(_fm_).is_in(in_nborhood)) | (pl.col(_to_).is_in(in_nborhood)))
        _df_map_ = {}
        for _node_ in node_to_cluster_center:
            if _node_ in in_nborhood: _df_map_[_node_] = _node_
            else:                     _df_map_[_node_] = -node_to_cluster_center[_node_]
        _df_ = _df_.with_columns(pl.col(_fm_).replace_strict(_df_map_).alias('__fm__'), 
                                 pl.col(_to_).replace_strict(_df_map_).alias('__to__'))
        _xy_   = _circles_[nborhood_i][0], _circles_[nborhood_i][1]
        _cd_r_ = _radius_
        #_svg_ = rt.chordDiagram(_df_, [('__fm__','__to__')], x_view=_xy_[0] - _cd_r_, y_view=_xy_[1] - _cd_r_, w=2*_cd_r_, h=2*_cd_r_, 
        #                        link_style='bundled', link_color=rt.co_mgr.getColor(nborhood_i), skeleton_algorithm='kmeans', draw_border=False, draw_circular_background=True)
        _svg_ = rt.chordDiagram(_df_, [('__fm__','__to__')], x_view=_xy_[0] - _cd_r_, y_view=_xy_[1] - _cd_r_, w=2*_cd_r_, h=2*_cd_r_, 
                                link_style='bundled', color_by='__fm__', link_color='vary', skeleton_algorithm='kmeans', draw_border=False, draw_circular_background=True)
        neighborhood_to_diagram[nborhood_i] = _svg_
        svg.append(_svg_._repr_svg_())

    svg.append('</svg>')
    return ''.join(svg)

rt.tile([chordRoutedGraph(k=10, draw_neighborhoods=True, w=800, h=800, **params), rt.link(w=512, h=512, **params)])

In [ ]:
_n_circles_    = 50
_circle_min_r_ = 10
_circle_max_r_ = 30
_circle_max_x_ = 400
_circle_max_y_ = 400
_circles_ = [(100 + (_circle_max_x_ - 200) * random.random(), 100 + (_circle_max_y_ - 200) * random.random(), random.randint(_circle_min_r_, _circle_max_r_)) for _ in range(_n_circles_)]
_svg_ = [f'<svg x="0" y="0" width="{_circle_max_x_}" height="{_circle_max_y_}">']
_svg_.append(f'<rect x="0" y="0" width="{_circle_max_x_}" height="{_circle_max_y_}" fill="white" />')
for i in range(len(_circles_)):
    _circle_ = _circles_[i]
    cx, cy, r = _circle_
    _svg_.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3"/>')
_svg_.append('</svg>')

_circles_uncrunched_ = rt.crunchCircles(_circles_)

_svg_uncrunched_ = [f'<svg x="0" y="0" width="{_circle_max_x_}" height="{_circle_max_y_}">']
_svg_uncrunched_.append(f'<rect x="0" y="0" width="{_circle_max_x_}" height="{_circle_max_y_}" fill="white" />')
for i in range(len(_circles_uncrunched_)):
    _circle_ = _circles_uncrunched_[i]
    cx, cy, r = _circle_
    _svg_uncrunched_.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3"/>')
_svg_uncrunched_.append('</svg>')
'''
rt.tile([''.join(_svg_), 
         f'<svg x="0" y="0" width="20" height="{_circle_max_y_}"><rect x="0" y="0" width="20" height="{_circle_max_y_}" fill="black" /></svg>', 
         ''.join(_svg_uncrunched_)])
'''

In [ ]:
_df1_ = pl.DataFrame({'fm':'a b c'.split(), 'to':'b c a'.split()})
_df2_ = pl.DataFrame({'fm':'d e f'.split(), 'to':'e f d'.split()})
rt.tile([rt.chordDiagram(_df1_, [('fm','to')], draw_labels=True, draw_background=False, w=128, h=128),         
         rt.chordDiagram(_df2_, [('fm','to')], draw_labels=True, draw_background=False, w=128, h=128)])

In [ ]:
svg_route = [f'<svg x="0" y="0" width="{768}" height="{256}">']
svg_route.append(f'<rect x="0" y="0" width="{768}" height="{256}" fill="white" />')
_radius_ = 64
_xy1_    = 164, 104
_cd1_ = rt.chordDiagram(_df1_, [('fm','to')], draw_labels=True, draw_background=False, x_view=_xy1_[0]-_radius_, y_view=_xy1_[1]-_radius_,  w=_radius_*2, h=_radius_*2, draw_border=False)
_xy2_    = 600, 160
_cd2_ = rt.chordDiagram(_df2_, [('fm','to')], draw_labels=True, draw_background=False, x_view=_xy2_[0]-_radius_, y_view=_xy2_[1]-_radius_, w=_radius_*2, h=_radius_*2, draw_border=False)
svg_route.append(_cd1_._repr_svg_())
svg_route.append(_cd2_._repr_svg_())

to_node = 'a'

_vecs1_ = _cd1_.entityPositions(to_node)[0].attachmentPointVecs()
_xyv1_   = _vecs1_[0][0], _vecs1_[0][1]
_uvv1_   = _vecs1_[0][2], _vecs1_[0][3]
_xyv1_t_ = _xy1_[0] - _radius_ + _xyv1_[0], _xy1_[1] - _radius_ + _xyv1_[1]
#svg_route.append(f'<line x1="{_xyv1_t_[0]}" y1="{_xyv1_t_[1]}" x2="{_xyv1_t_[0] + 30*_uvv1_[0]}" y2="{_xyv1_t_[1] + 30*_uvv1_[1]}" stroke="black" stroke-width="3" />')

fm_node = 'd'

_vecs2_ = _cd2_.entityPositions(fm_node)[0].attachmentPointVecs()
_xyv2_   = _vecs2_[0][0], _vecs2_[0][1]
_uvv2_   = _vecs2_[0][2], _vecs2_[0][3]
_xyv2_t_ = _xy2_[0] - _radius_ + _xyv2_[0], _xy2_[1] - _radius_ + _xyv2_[1]
#svg_route.append(f'<line x1="{_xyv2_t_[0]}" y1="{_xyv2_t_[1]}" x2="{_xyv2_t_[0] + 30*_uvv2_[0]}" y2="{_xyv2_t_[1] + 30*_uvv2_[1]}" stroke="black" stroke-width="3" />')

_other_circle_ = (384, 128, 48)
svg_route.append(f'<circle cx="{_other_circle_[0]}" cy="{_other_circle_[1]}" r="{_other_circle_[2]}" stroke="black" stroke-width="3" fill="none" />')

_circle_geoms_ = [(_xy1_[0], _xy1_[1], _radius_),(_xy2_[0], _xy2_[1], _radius_), _other_circle_]
_push_out_     = 18
_entry_pt_     = ( _xyv1_t_[0] + _push_out_*_uvv1_[0], _xyv1_t_[1] + _push_out_*_uvv1_[1], 0)
_exit_pts_     = [(_xyv2_t_[0] + _push_out_*_uvv2_[0], _xyv2_t_[1] + _push_out_*_uvv2_[1], 1)]
_tuple_ = rt.circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_)
_coords_ = _tuple_[0][0]
d =f'M {_coords_[0][0]} {_coords_[0][1]} '
for i in range(1, len(_coords_)): d += f'L {_coords_[i][0]} {_coords_[i][1]} '

svg_route.append(f'<path d="{d}" fill="none" stroke="black" stroke-width="3" />')

svg_route.append('</svg>')
rt.tile([''.join(svg_route)])